# Het extraheren van data uit webforums

In deze tutorial doe je ervaring op in het parseren van Forum data in python. We beginnen met het importeren van handige libraries:

In [1]:
import sys
#sys.path.append('/home/vagrant/webdata_collect/scripts/ADNEXT_collect/')
from bs4 import BeautifulSoup
import requests
import re
from forum_xml_classes import Forum, Thread, Post

<em>BeautifulSoup</em> is de ster van het verhaal. Deze library maakt het mogelijk om een parseringsscript te schrijven toegespitst op de specifieke tags die gebruikt worden in een webpagina. <em>requests</em> wordt gebruikt om webdata binnen te halen, <em>re</em> is een library voor het matchen van reguliere expressies en <em>forum_xml_classes</em> is een 'lokale' library om de data in weg te schrijven. 

We beginnen met het binnenhalen van een pagina met forum posts. Het testforum is forum.body-fitness.nl, een forum voor bodybuilding, waar mogelijk informatie over illegale voedingssupplementen wordt gedeeld. Het commando om de broncode van een webpagina op te halen is 'requests.get':

In [2]:
url='http://forum.body-fitness.nl/Hematocriet-verhogen-m719672.aspx'
response=requests.get(url)

Nu we de webpagina hebben binnengehaald, kunnen we inspecteren waaruit dit bestaat. Hiervoor kan het commando 'content' gebruikt worden:

In [3]:
response.content

b'\r\n\r\n<!DOCTYPE html>\r\n\r\n<!--[if lt IE 7 ]> <html class="ie6" > <![endif]-->\r\n<!--[if IE 7 ]>    <html class="ie7" > <![endif]-->\r\n<!--[if IE 8 ]>    <html class="ie8" > <![endif]-->\r\n<!--[if IE 9 ]>    <html class="ie9" > <![endif]-->\r\n<!--[if (gt IE 9)|!(IE)]><!--> <html class="" > <!--<![endif]-->\r\n<head id="Head1"><title>\r\n\tHematocriet verhogen | Body & Fitness - Bodybuilding en Fitness forum\r\n</title>\r\n      <script type="text/javascript">\r\n          try {\r\n              document.execCommand("BackgroundImageCache", false, true);\r\n          } catch (err) { }\r\n      </script>    \r\n\r\n    \r\n    \r\n    \r\n\r\n\r\n        \r\n    <script type="text/javascript">\r\n\r\n        var _gaq = _gaq || [];\r\n        _gaq.push([\'_setAccount\', \'UA-684806-2\']);\r\n        \r\n        _gaq.push([\'_trackPageview\']);\r\n        \r\n        (function () {\r\n            var ga = document.createElement(\'script\'); ga.type = \'text/javascript\'; ga.async 

Nogal onoverzichtelijk misschien. Deze html-code is natuurlijk niet bestemd voor mensenogen. Om hier de gewenste data uit te halen, zullen we een idee moeten krijgen van de structuur. Er moet een <em>tag</em> zijn waarbinnen unieke posts zijn ingekapseld. Tags zijn te herkennen aan eenheden binnen twee gehoekte haken ('<' en '>'). De eerste stap is om deze specifieke tag te identificeren. Het kan daarbij helpen om de url in de browser te inspecteren, en vervolgens met CTRL-F in de html-code te zoeken naar een uniek stukje tekst in een post. Bekijk vervolgens welke tags hieromheen staan. Probeer de tags te vinden die niet alleen de tekst omvatten, maar ook informatie als de auteur en datum. 

Gelukt? Met deze kennis kunnen we beginnen met selecteren van unieke posts uit de html-code. Om te beginnen gaan we het bestand 'soupen':

In [4]:
soup = BeautifulSoup(response.content,'html.parser')

Door de inhoud van het bestand om te vormen tot een BeautifulSoup object, kunnen we gemakkelijk tags selecteren. Met slechts een paar stappen hebben we de gewenste informatie uit het bestand gehaald. Een van de belangrijkste commando's is 'find_all'. Daarmee kan alle inhoud binnen een bepaalde <em>tag</em> binnengehaald worden. De tag kan verder gespecificeerd worden, door haar attributen te specificeren. De tags waarmee posts gekenmerkt worden op deze pagina geven we cadeau: 

In [6]:
messages = soup.find_all('td', { 'class':'msgtable item '})

Misschien had je het zelf ook al gezien in de markup: complete posts worden gekarakteriseerd met de tag 'td' en met een attribuut 'class' die steeds de waarde 'msgtable item ' heeft. Attributen en waarden kunnen aan deze zoekopdracht meegegeven worden als key en value van een python dictionary. Logischerwijs kun je tegelijkertijd meerdere restricties specificeren, mocht dit nodig zijn. 

Als het goed is hebben we nu alle posts apart gezet. Om te testen is het handig om het aantal te tellen:

In [7]:
# print het aantal posts
print(len(messages))

15


Verder zijn we benieuwd naar de inhoud van een enkele post. Deze moeten we gaan extraheren. Dit kan gedaan worden door een enkele post te selecteren, en aan te roepen met de extensie '.contents':

In [8]:
# selecteer een enkele post en schrijf toe aan een variabele
testmessage = messages[0]
# print de inhoud van deze post
testmessage.contents

['\n', <div class="altItem essential authorcontent">
 <div class="essentialAuthorLine">
 <div class="essentialAvatar" id="Avatar719672">
 <img class="midAvatar" src="/thumb.axd/145_15182/BABDCDEC80564607ACCA9A81E7313B38.jpg" style="" title="Marathonman"/>
 </div>
 <div class="left">
 <a class="left titlehead " data-isfriend="false" data-isignored="false" data-isownpost="false" data-isrecycled="false" data-login="Marathonman" data-mem="15182" data-messageid="719672" data-showpmlink="false" data-viewerisguest="true" href="/Profile/15182/">Marathonman </a>
 <div class="divider"></div>
 <span class="left msgLabel tmUserTitle ">Heavy Weight</span>
 </div>
 </div>
 </div>, '\n', <div class="item essential msgcontent">
 <span id="msgNum1"></span>
 <div class=" lPadding10">
 <img class="msgIcon" src="/app_themes/Progressive/image/mIcons/m1.gif"/>
 <div class="msgSubjectLine">
 <span class="msgInfo">
 <span class="msgDate " data-mid="719672" id="date719672">
 <span class="performdateformat" dat

Als het goed is zie je een hele serie tags, met ergens in het midden de daadwerkelijke tekst van het bericht. Laten we beginnen om deze tekst te extraheren. Hiervoor kun je de tag selecteren die de tekst direct omringt. De tag zal behoorlijk algemeen zijn, maar met het specificeren van de attribuut kunnen we toch de juiste tag aanwijzen. In plaats van het commando 'find_all' gebruiken we nu 'find_next'. Hiermee kan de eerstvolgende markup die voldoet aan de opgegeven restrictie opgehaald worden:

In [9]:
# extraheer de tekst van de markup van een enkele post ( posttext = testmessage.find_next(...) )
posttext = testmessage.find_next('div', { 'class':'msgSection'})

Nu we de tags met de text hebben geselecteerd, kunnen we hier de text uithalen. Hiervoor kan simpelweg de extensie '.text' gebruikt worden:

In [10]:
posttext.text

'\nHematocriet verhogen\r\n                Misschien iemand een idee of er iets is dat op legale manier je hematocriet merkbaar kan verhogen?\n'

Deze output is nog niet erg mooi. Laten we eerst de witte ruimte weghalen. Hiervoor gebruiken we de 're' library:

In [13]:
textparts = re.split(r'\s{2,}',posttext.text)

Om ook de laatste witte ruimte weg te halen kunnen het commando 'strip()' gebruiken. We houden dan een paar paragrafen van het bericht over:

In [14]:
paragraphs = [paragraph.strip() for paragraph in textparts]
print(paragraphs)

['Hematocriet verhogen', 'Misschien iemand een idee of er iets is dat op legale manier je hematocriet merkbaar kan verhogen?']


Nu we de tekst hebben, kunnen we de meta-data van het bericht ophalen. Te beginnen met de gebruikersnaam van de auteur. Deze kan op dezelfde manier opgehaald worden als de tekst: zoek naar de gebruikersnaam in de volledige contents van het bericht, en identificeer de tag en een specifieke attribuutwaarde. Haal het vervolgens op met 'find_next'. 

In [28]:
# extraheer de gebruikersnaam en schrijf naar de variabele 'authorname'
authorname = testmessage.find_next('a', { 'class':'left titlehead '}).text
# print als check de variabele
print(authorname)

Marathonman 


Naast de tekst en auteursnaam bevatten forumposts bijna altijd een datum en index. Beiden worden in dit forum gespecificeerd binnen een tag 'span'. Daarom zullen we nadere kenmerken in de attributen moeten specificeren. Eerst een check welke varianten er zijn in deze post:

In [18]:
spans = testmessage.find_all('span')
for span in spans:
    print('Span',span.attrs)

Span {'class': ['left', 'msgLabel', 'tmUserTitle', '']}
Span {'id': 'msgNum1'}
Span {'class': ['msgInfo']}
Span {'data-mid': '719672', 'class': ['msgDate', ''], 'id': 'date719672'}
Span {'class': ['performdateformat'], 'data-format': 'ddd dd-MMM-yy HH:mm', 'data-date': '2013/12/05 01:47:46'}
Span {'class': ['ratingResult'], 'id': 'ratingResult719672'}
Span {'class': ['wideOptionSep']}


De index van de post wordt gekenmerkt door een attribuut 'id'. Het probleem is alleen dat er nog een andere tag is met deze attribuut. Omdat de waarde van de index gegeven wordt als waarde van de attribuut, is dit het enige kenmerkende voor de index. Gelukkig wordt iedere index beschreven met 'msgNum'. We kunnen de index dus identificeren via reguliere expressies: 

In [23]:
with_id = [span for span in spans if 'id' in span.attrs]
for span in with_id:
    if re.match(r'^msgNum\d+',span['id']):
        messageindex = int(span['id'][6:])
        break
print(messageindex)

1


In [22]:
with_id = [span for span in spans if 'id' in span.attrs]
[x.contents for x in with_id]

[[],
 ['\n',
  <span class="performdateformat" data-date="2013/12/05 01:47:46" data-format="ddd dd-MMM-yy HH:mm">2013/12/05 01:47:46</span>,
  '\n'],
 ['0']]

Voor het extraheren van de datum kan weer het commando 'find_next' gebruikt worden. Omdat de attribuutwaarde als lijst gegeven is (binnen geblokte haken), zul je deze lijst met inhoud als attribuutwaarde moeten specificeren. Een ander verschil is dat de datum binnen de tag gegeven wordt. We hebben hieronder al de code gegeven om deze informatie op te halen. Alleen de juiste tag moet nog gespecificeerd worden zoals hiervoor:

In [20]:
# Selecteer de unieke tag waarin datuminformatie gegeven wordt, en schrijf naar de variabele 'datetag'
datetag = testmessage.find_next('span', { 'class':['performdateformat']})
# Extraheer de datum en tijd uit deze tag
datetime = datetag['data-date']
print(datetime)

2013/12/05 01:47:46


Tot slot kunnen we het id van het bericht extraheren. Deze is gegeven als attribuut van de hoofdtag:

In [24]:
postid = testmessage['id'][3:]

Naast de geëstraheerde informatie, zou een post informatie kunnen bevatten over haar populariteit, en posts waar het een reactie op is. Deze laten we hier buiten beschouwing. Met de huidige informatie kunnen we een 'Post' object aanmaken, waarmee de informatie in overzichtelijk XML gezet kan worden. 

In [29]:
post = Post(postid, authorname, datetime, paragraphs, messageindex, '-', '-', '-')
post.returnXML()

'<post id="719672">\n<author>Marathonman </author>\n<timestamp>2013/12/05 01:47:46</timestamp>\n<postindex>1</postindex>\n<parentid>-</parentid>\n<body>\n<paragraph>Hematocriet verhogen</paragraph>\n<paragraph>Misschien iemand een idee of er iets is dat op legale manier je hematocriet merkbaar kan verhogen?</paragraph>\n</body>\n<upvotes>-</upvotes>\n<downvotes>-</downvotes>\n</post>\n'

Nu we uit dit ene bericht de gewenste informatie hebben gehaald, kunnen we proberen alle 15 de berichten te parseren. Hiervoor schrijven we een serie functies waarmee ieder bericht doorlopen is. De eerste geven we cadeau. De rest is een opdracht. Als het werkt heb je voor alle 15 berichten de informatie-eenheden geëxtraheerd.

In [32]:
def extract_index(souped_post):
    spans = testmessage.find_all('span')
    with_id = [span for span in spans if 'id' in span.attrs]
    for span in with_id:
        if re.match(r'^msgNum\d+',span['id']):
            messageindex = int(span['id'][6:])
            break
    return messageindex

def extract_postid(souped_post):
    postid = souped_post['id'][3:]
    return postid
    
def extract_paragraphs(souped_post):
    posttext = souped_post.find_next('div', { 'class':'msgSection'})
    textparts = re.split(r'\s{2,}',posttext.text)
    paragraphs = [paragraph.strip() for paragraph in textparts]
    return paragraphs
    
def extract_author(souped_post):
    authorname = souped_post.find_next('a', { 'class':'left titlehead '}).text
    return authorname
    
def extract_datetime(souped_post):
    datetag = souped_post.find_next('span', { 'class':['performdateformat']})
    datetime = datetag['data-date']
    return datetime
    
def parse_forumpost(souped_post):
    index = extract_index(souped_post)
    postid = extract_postid(souped_post)
    paragraphs = extract_paragraphs(souped_post)
    datetime = extract_datetime(souped_post)
    author = extract_author(souped_post)
    post = Post(postid,author,datetime,paragraphs,index,'-','-','-')
    return post

thread = Thread('1', 'Hematocriet verhogen', '-', '-')
for i,message in enumerate(messages):
    parsed = parse_forumpost(message)
    print(i,parsed.returnXML())
    thread.addPost(parsed)

print(thread.getNrOfPosts())

0 <post id="719672">
<author>Marathonman </author>
<timestamp>2013/12/05 01:47:46</timestamp>
<postindex>1</postindex>
<parentid>-</parentid>
<body>
<paragraph>Hematocriet verhogen</paragraph>
<paragraph>Misschien iemand een idee of er iets is dat op legale manier je hematocriet merkbaar kan verhogen?</paragraph>
</body>
<upvotes>-</upvotes>
<downvotes>-</downvotes>
</post>

1 <post id="719679">
<author>teddybeerke </author>
<timestamp>2013/12/09 14:05:18</timestamp>
<postindex>1</postindex>
<parentid>-</parentid>
<body>
<paragraph></paragraph>
<paragraph>Rode bietensap</paragraph>
<paragraph></paragraph>
</body>
<upvotes>-</upvotes>
<downvotes>-</downvotes>
</post>

2 <post id="719683">
<author>Marathonman </author>
<timestamp>2013/12/09 21:39:38</timestamp>
<postindex>1</postindex>
<parentid>-</parentid>
<body>
<paragraph></paragraph>
<paragraph>bietjes!</paragraph>
<paragraph>o shit.. merkbaar</paragraph>
<paragraph></paragraph>
</body>
<upvotes>-</upvotes>
<downvotes>-</downvotes>


Als het gelukt is kunnen we tot slot het bestand wegschrijven. Uiteindelijk kan dit proces geautomatiseerd worden om een volledig forum te parseren.  

In [ ]:
outfile = '/home/vagrant/webdata_collect/data/thread_body-fitness.xml'
with open(outfile,'w',encoding = 'iso-8859-1', errors = 'replace') as out:
    thread.printXML(out)

Helaas gebruikt ieder forum weer andere tags. Dus als we de data uit een nieuw forum willen extraheren zijn we weer terug bij af. De vaardigheid om de belangrijke structuursignalen in een html-bestand te vinden en via BeautifulSoup toe te passen kan dus erg nuttig zijn. 

In [ ]:
# BONUSOPDRACHT: Probeer dezelfde informatie-eenheden als hiervoor te extraheren uit een pagina in het bodybuilding-forum
bodybuildingtestpage = 'http://forum.bodybuilding.nl/topics/vitamines-en-pijn.402738/index.html'
